# TODOs

- [x] Figure out why resumption is not working
- Verify that loss function is implemented right 
- record / plot loss better and easy background training
- [X] Research more QLoRA
  - [ ] make sure that QLoRA is actually happening..
  - RTFP
  - [X] Find good solid example code with transformers
  - Determine QLoRA parameters to experiment with and how
    - datatype
    - what bits
    - r
    - dropout
    - target_modules
    - lora alpha
- Determine training parameters to experiment with
  - AdamW RTFP
  - tune adamW starting learning rate?
  - Batch size. Should it vary over time?
  - 
- get complicated and weird
  - explore mining hard negatives
  - explore synthesizing hard negatives
  - synthesize more varied data
  - generated data off of a target domain (easy-on fine tuning)
- benchmark
  - add inferrence code path
  - figure out how to run model against an mteb benchmark
- update code to be able to run it distributed in the cloud?
  - related to deepspeed and accelerate
    - https://arxiv.org/abs/2104.07857
    - https://huggingface.co/docs/accelerate/usage_guides/deepspeed
- [x] get a bigger graphics card

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import train
seed = 42
base_model, tokenizer = train.load_model()
model = train.load_model_for_training(base_model)

dataset = train.prepare_dataset(tokenizer, seed)
# Access the new train and test datasets
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

from transformers import TrainingArguments, IntervalStrategy

from train import TinyEmbedTrainer

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    log_level="info",
    logging_strategy=IntervalStrategy.STEPS,
    gradient_checkpointing=True,
    logging_steps=5,
    save_steps=1000,
    seed=seed,
)

trainer = TinyEmbedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)
trainer.train(resume_from_checkpoint=False)

***** Running training *****
  Num examples = 163,636
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 122,727
  Number of trainable parameters = 5,902,336
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/mnt/freezer/dev/tinyllama-embed/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
5,0.654800
10,0.681800
15,0.695000
20,0.692000
25,0.659200
30,0.685400
35,0.652300
40,0.655700
45,0.614400
